In [1]:
import pandas as pd
import numpy as np
import os
import json

In [2]:
files_coord_curves = [x for x in os.listdir('./') if x.startswith('coord_curve')]

In [3]:
files_coord_curves

['coord_curve_S3_S5.csv',
 'coord_curve_S3_S4.csv',
 'coord_curve_S0_S1.csv',
 'coord_curve_S0_S2.csv',
 'coord_curve_S0_S3.csv']

In [4]:
dict_df = dict() #dictionary for dataframes of branches
dict_len = dict() #dictionary for lengths of branches
dict_pseudotime=dict()
for x in files_coord_curves:
    len_x = 0
    list_len_x = [0]
    df_file_x = pd.read_csv(x,sep='\t')
    for i in range(df_file_x.shape[0]-1):
        len_x += np.linalg.norm(df_file_x.iloc[i+1,]-df_file_x.iloc[i,])
        list_len_x.append(len_x)
    dict_len[x[-9:-4]] = len_x
    dict_df[x[-9:-4]] = df_file_x
    dict_pseudotime[x[-9:-4]] = list_len_x

In [5]:
dict_len

{'S3_S5': 0.1927126961060231,
 'S3_S4': 0.04802614380489234,
 'S0_S1': 0.1283525141658179,
 'S0_S2': 0.05332331986937152,
 'S0_S3': 0.021015061188881107}

In [6]:
len_max = max(dict_len.values())

In [7]:
step = len_max/100

In [8]:
list_curves = []
for x in dict_df.keys():
    df_file_x = dict_df[x]
    
    pseudotime_new = np.arange(dict_pseudotime[x][0],dict_pseudotime[x][-1],step)
    c1 = np.interp(pseudotime_new,dict_pseudotime[x],df_file_x['0'])
    c2 = np.interp(pseudotime_new, dict_pseudotime[x],df_file_x['1'])  
    c3 = np.interp(pseudotime_new, dict_pseudotime[x],df_file_x['2'])
        
    if(pseudotime_new[-1]<dict_len[x]):
        c1 = np.append(c1,df_file_x['0'].iloc[-1])
        c2 = np.append(c2,df_file_x['1'].iloc[-1])  
        c3 = np.append(c3,df_file_x['2'].iloc[-1])  
        pseudotime_new = np.append(pseudotime_new,dict_len[x])  

    for i in range(c1.shape[0]):
        dict_coord_curves = dict()
        dict_coord_curves['branch_id'] = x
        dict_coord_curves['pseudotime'] = pseudotime_new[i]
        dict_coord_curves['x'] = c1[i]
        dict_coord_curves['y'] = c2[i]
        dict_coord_curves['z'] = c3[i]
        list_curves.append(dict_coord_curves)

In [9]:
with open('curves.json', 'w') as f:
    json.dump(list_curves, f)